# **Practical Exam Machine Learning Lab - Hard Task**
Done by: Siddharth Sudhakar (25901335)

Dataset: DS9 Advertising Spend vs Sales
https://www.kaggle.com/datasets/thorgodofthunder/tvradionewspaperadvertising

Task:
* Easy (Linear Regression) - DS9: Train/test split; MAE.
* Moderate (Gradient Descent) - DS9: Implement GD; show learning-rate sensitivity (3 eta
values).
* Hard (Closed-form vs GD) - DS9: Derive normal-equation solution using numpy (no sklearn).
Compare coefficients and MAE vs GD vs sklearn

## **Closed-Form Task**

In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

In [4]:
from sklearn.linear_model import LinearRegression

In [5]:
data = pd.read_csv('Advertising.csv')

### **Normal Equation implementation using NumPy**

In [6]:
class LinearRegressionNormalEqn:
    def fit(self, X, y):
        X_b = np.c_[np.ones((X.shape[0], 1)), X]
        theta = np.linalg.inv(X_b.T @ X_b) @ X_b.T @ y
        self.b_ne = theta[0]
        self.w_ne = theta[1:]

    def predict(self, X):
        return X @ self.w_ne + self.b_ne

In [7]:
feature_col=['TV']
X = data[feature_col]
y = data.Sales

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)

In [9]:
ne_model = LinearRegressionNormalEqn()
ne_model.fit(X_train, y_train)

y_pred_ne = ne_model.predict(X_test)

mae_ne=mean_absolute_error(y_test, y_pred_ne)
print("Mean Absolute Error of Model with lr=0.1:", mae_ne)

r2_ne=r2_score(y_test, y_pred_ne)
print("R2 Score of Model with lr=0.1:", r2_ne)

Mean Absolute Error of Model with lr=0.1: 1.9502948931650081
R2 Score of Model with lr=0.1: 0.8025613034236982


### **Coefficient Comparison**

In [10]:
sk_model = LinearRegression()
sk_model.fit(X_train, y_train)
y_pred_sk = sk_model.predict(X_test)
maeSK=mean_absolute_error(y_test, y_pred_sk)

In [11]:
class LinearRegressionGD:
    def __init__(self, lr=0.01, epochs=1000):
        self.lr = lr
        self.epochs = epochs

    def fit(self, X, y):
        n_samples, n_features = X.shape

        self.w = np.zeros(n_features)
        self.b = 0.0

        for _ in range(self.epochs):
            y_pred = X @ self.w + self.b

            # Gradients
            error = y_pred - y
            dw = (2 / n_samples) * (X.T @ error)
            db = (2 / n_samples) * np.sum(error)

            # update
            self.w -= self.lr * dw
            self.b -= self.lr * db

    def predict(self, X):
        return X @ self.w + self.b

In [12]:
X_mean = X.mean(axis=0)
X_std = X.std(axis=0)

X_scaled = (X - X_mean) / X_std

In [13]:
X_train_gd, X_test_gd, y_train_gd, y_test_gd = train_test_split(X_scaled, y, test_size = 0.2, random_state=42)

In [14]:
gd_model = LinearRegressionGD(lr=0.001, epochs=5000)
gd_model.fit(X_train_gd, y_train_gd)
print("Weight and bias of Model with lr=0.05:",gd_model.w, gd_model.b)
y_pred_gd = gd_model.predict(X_test_gd)
maeGD=mean_absolute_error(y_test_gd, y_pred_gd)

Weight and bias of Model with lr=0.05: TV    4.76341
dtype: float64 15.164822541421264


In [15]:
print("=== Coefficients ===")
print("Normal Equation:")
print("Bias:", ne_model.b_ne, "Weights:", ne_model.w_ne)

print("\nGradient Descent:")
print("Bias:", gd_model.b, "Weights:", gd_model.w)

print("\nsklearn:")
print("Bias:", sk_model.intercept_, "Weights:", sk_model.coef_)

print("\n=== MAE Comparison ===")
print("Normal Equation MAE:", mae_ne)
print("Gradient Descent MAE:", maeGD)
print("sklearn MAE:", maeSK)

=== Coefficients ===
Normal Equation:
Bias: 7.0071084282418425 Weights: [0.05548294]

Gradient Descent:
Bias: 15.164822541421264 Weights: TV    4.76341
dtype: float64

sklearn:
Bias: 7.007108428241848 Weights: [0.05548294]

=== MAE Comparison ===
Normal Equation MAE: 1.9502948931650081
Gradient Descent MAE: 1.9502301726645626
sklearn MAE: 1.9502948931650088
